# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [33]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time

Setting path to train and test data.

In [34]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32
BATCH_SIZE = 100

Loading train data.

In [35]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [36]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [37]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [ ]:
train_labels, train_images = load_train_data(TRAIN_DATA_DIR)
val_labels, val_images = load_test_data(TEST_DATA_DIR)

In [ ]:
print("Total training images {}".format(len(images)))
print("Total testing images {}".format(len(test_images)))

Creating bar chart for dataset Visualization.

In [ ]:
data = {x:train_labels.count(x) for x in train_labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel('Number of examples')
plt.title('Training dataset')
plt.show()

In [ ]:
def lerelu(x):
    return tf.maximum(x, x * 0.02)

In [ ]:
def conv2d(input_layer, filters):
    return tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=(5, 5),
        padding='same',
        activation=tf.nn.elu)

def pooling(input_layer):
    return tf.layers.max_pooling2d(
        inputs=input_layer, 
        pool_size=(2, 2), 
   EPOCH     strides=(2, 2),
        padding='same')

with tf.device('/device:GPU:0'):    
    images = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3])
    labels = tf.placeholder(tf.int64, [None])
    
    conv1 = conv2d(images, 8)
    pool1 = pooling(conv1)
    
    conv2 = conv2d(pool1, 12)
    pool2 = pooling(conv2)
    
    conv3 = conv2d(pool2, 16)
    pool3 = pooling(conv3)
   
    flat = tf.contrib.layers.flatten(pool3)
    
    logits = tf.contrib.layers.fully_connected(flat, 44, tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1)
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [ ]:
session = tf.Session()
session.run(init)

In [ ]:
train_labels_arr = np.array(train_labels)
train_images_arr = np.array(train_images)

In [ ]:
def validation(train_images, train_labels, val_images, val_labels):
    train_acc = session.run(accuracy,
                    {images: train_images,
                    labels: train_labels})
    val_acc = session.run(accuracy,
                    {images: val_images,
                    labels: val_labels})
    return train_acc, val_acc

In [ ]:
def train_model(training_images, training_labels, val_images, val_labels, steps):
    t_start = time.time()
    
    for i in range(steps + 1):
        indexes = np.random.choice(np.arange(len(train_images)), BATCH_SIZE, replace = False)
        batch_images = train_images_arr[indexes]
        batch_labels = train_labels_arr[indexes]
        loss_value = session.run([optimizer, loss],
                                 {images: batch_images,
                                 labels: batch_labels})[1]
        if (steps % 50 == 0):
            training_acc, val_acc = validation(batch_images, batch_labels, val_images, val_labels)
            
train_model(train_images, train_labels, val_images, val_labels, 100)


In [32]:
session.close()